# RNN Classifier con PyTorch Lightning
## Trabajo Computacional 2

Este notebook implementa un clasificador RNN para predecir el origen de nombres utilizando PyTorch Lightning.

### Objetivos
- Implementar un modelo RNN, LSTM y GRU para la tarea de clasificación usando PyTorch Lightning
- Modularizar el código en componentes reutilizables:
  - `model.py`: Definición del modelo RNN
  - `datamodule.py`: Manejo de datos y preprocessing
  - `train.py`: Loop de entrenamiento y evaluación

## Instalación y configuración

Primero instalamos las dependencias necesarias e importamos las bibliotecas.

In [8]:
# Instalar PyTorch Lightning si no está instalado
# !pip install lightning

In [9]:
import os
import torch
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import numpy as np
import time

# Importar módulos personalizados
from utils.model import RNNClassifier
from utils.datamodule import RNNDataset, RNNDataModule, n_letters
from utils.eval import evaluate_model, plot_confusion_matrix, predict_name_origin

from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import TQDMProgressBar, RichProgressBar
torch.manual_seed(47)

print(f"PyTorch version: {torch.__version__}")
print(f"PyTorch Lightning version: {pl.__version__}")
print(f"Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

PyTorch version: 2.3.0+cu121
PyTorch Lightning version: 2.5.3
Device: CUDA


## Preparación de datos

Descargamos y preparamos los datos de nombres si no están disponibles.

In [10]:
# Descargar datos si no existen
if not os.path.exists('./data'):
    print("Descargando datos...")
    !wget https://download.pytorch.org/tutorial/data.zip
    !unzip data.zip
    !rm data.zip
    print("Datos descargados exitosamente!")
else:
    print("Los datos ya están disponibles.")

Los datos ya están disponibles.


### Distribución de datos y balanceo

Al cargar e inspeccionar los datos, estos se encontraban muy desbalanceados. Para poder definir las épocas adecuadamente se hizo un sobremuestreo de las clases con menos datos y con esto tener una distribución más equitativa de los datos

In [11]:
# Cargamos el dataset
dataset = RNNDataset(data_path="./data/names/", balanced=True, verbose=True)

Cargando datos desde ./data/names/...


Polish: 139 ejemplos
Greek: 203 ejemplos
Chinese: 268 ejemplos
Scottish: 100 ejemplos
Italian: 709 ejemplos
German: 724 ejemplos
Dutch: 297 ejemplos
Arabic: 2000 ejemplos
Portuguese: 74 ejemplos
English: 3668 ejemplos
Russian: 9408 ejemplos
French: 277 ejemplos
Czech: 519 ejemplos
Japanese: 991 ejemplos
Spanish: 298 ejemplos
Korean: 94 ejemplos
Vietnamese: 73 ejemplos
Irish: 232 ejemplos

Balanceando dataset...
Polish - nuevo tamaño: 9313
Greek - nuevo tamaño: 9338
Chinese - nuevo tamaño: 9380
Scottish - nuevo tamaño: 9400
Italian - nuevo tamaño: 9217
German - nuevo tamaño: 8688
Dutch - nuevo tamaño: 9207
Arabic - nuevo tamaño: 8000
Portuguese - nuevo tamaño: 9398
English - nuevo tamaño: 7336
French - nuevo tamaño: 9141
Czech - nuevo tamaño: 9342
Japanese - nuevo tamaño: 8919
Spanish - nuevo tamaño: 9238
Korean - nuevo tamaño: 9400
Vietnamese - nuevo tamaño: 9344
Irish - nuevo tamaño: 9280


In [12]:
categories = dataset.categories

print(f"Número total de categorías: {len(categories)}")
print(f"Categorías disponibles: {categories}")

print(f"\nEjemplos de entrenamiento:")
for i in range(5):
    category, line, category_tensor, line_tensor = dataset.get_random_sample()
    print(f"category = {category} / line = {line}")


Número total de categorías: 18
Categorías disponibles: ['Polish', 'Greek', 'Chinese', 'Scottish', 'Italian', 'German', 'Dutch', 'Arabic', 'Portuguese', 'English', 'Russian', 'French', 'Czech', 'Japanese', 'Spanish', 'Korean', 'Vietnamese', 'Irish']

Ejemplos de entrenamiento:
category = Czech / line = Pear
category = Portuguese / line = Serafim
category = Portuguese / line = Salazar
category = Arabic / line = Khoury
category = Greek / line = Makricosta


## Entrenamiento del modelo

Ahora entrenamos el modelo RNN usando PyTorch Lightning. El entrenamiento esta dentro de un script de python separado para poder ejecutarlo fuera del kernel interactivo del presente notebook

In [ ]:
import subprocess
import sys

# Ejecuta el script de entrenamiento usando subprocess
script_path = "rnn-training.py"

# Verificar que el script existe
if not os.path.exists(script_path):
    print(f"Error: No se encontró el archivo {script_path}")
    exit(1)

print("Iniciando entrenamiento del modelo RNN...")

try:
    # Ejecutar el script con subprocess
    process = subprocess.Popen(
        [sys.executable, script_path],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
        bufsize=1
    )
    
    # Mostrar la salida en tiempo real
    for line in process.stdout:
        print(line.rstrip())
    
    # Esperar a que termine el proceso
    process.wait()
    
    if process.returncode == 0:
        print("Entrenamiento completado exitosamente!")
    else:
        print("\n" + "=" * 60)
        print(f"Error durante el entrenamiento. Código de salida: {process.returncode}")
        
except Exception as e:
    print(f"Error al ejecutar el script: {str(e)}")

Iniciando entrenamiento del modelo RNN...


PyTorch version: 2.3.0+cu121
PyTorch Lightning version: 2.5.3
Device: CUDA
Configuración del entrenamiento:
  hidden_size: 128
  learning_rate: 0.00125
  n_epochs: 100
  batch_size: 32
  num_workers: 40
Probando modelo base: rnn
Creando instancia de modelo RNN (rnn)...
Modelo creado exitosamente!
Arquitectura del modelo:
  - Input size: 57 (caracteres)
  - Hidden size: 128
  - Output size: 18 (categorías)
  - Learning rate: 0.00125
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Cargando modelo desde checkpoint: ./checkpoints/rnn/epoch=48-step=47187.ckpt
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/4
PyTorch version: 2.3.0+cu121
PyTorch Lightning version: 2.5.3
Device: CUDA
Configuración del entrenamiento:
  hidden_size: 128
  learning_rate: 0.00125
  n_epochs: 100
  batch_size: 32
  num_workers: 40
Probando modelo base: rnn
Creando instancia de modelo RNN (rnn)...
PyTorch version: 2.3.0+cu121
PyTorch Lightning 

## Evaluación del modelo

Evaluamos el modelo entrenado y visualizamos los resultados.

In [ ]:
# Evaluar el modelo (como en el original)
print("Evaluando el modelo...")
# base_models = ["rnn", "lstm", "gru"]
base_models = ["rnn"]  # Para simplificar, solo evaluamos el modelo RNN

for base_model in base_models:
    print(f"\nEvaluando modelo: {base_model.upper()}")

    # Cargar el mejor checkpoint basado en validación
    best_checkpoint = "./checkpoints/{base_model}/epoch=48-step=47187.ckpt"
    model = RNNClassifier.load_from_checkpoint(best_checkpoint)

    trainer = pl.Trainer(
        max_epochs=1,
        callbacks=[],
        accelerator="auto",  # Uses GPUs or TPUs if available
        devices="auto",  # Uses all available GPUs/TPUs if applicable
        deterministic=False,
        log_every_n_steps=10,
    )

    trainer.test(model)
    # accuracy, confusion_matrix, categories = evaluate_model(model, data_module, n_samples=1000)

    # print(f"Precisión: {accuracy:.4f}")
    # print(f"Número de categorías: {len(categories)}")

    # # Visualizar matriz de confusión
    # plot_confusion_matrix(confusion_matrix, categories, 'Matriz de Confusión - RNN Classifier')

Evaluando el modelo...


NameError: name 'model' is not defined

## Predicciones

Probamos el modelo con algunos nombres de ejemplo.

In [ ]:
# Probar predicciones con nombres de ejemplo
# test_names = ['Dovesky', 'Jackson', 'Satoshi', 'Rodriguez', 'Mueller', 'Li', 'Nakamura', 'Smith']

# print("Predicciones del modelo:")
# print("=" * 50)

# for name in test_names:
#     predictions = predict_name_origin(model, name, categories, n_predictions=3)
#     print()  # Línea en blanco entre predicciones

Predicciones del modelo:

> Dovesky
(-0.72) Czech
(-0.82) Russian
(-3.01) English


> Jackson
(-0.16) Scottish
(-2.07) English
(-4.76) Russian


> Satoshi
(-0.39) Japanese
(-1.69) Italian
(-2.68) Portuguese


> Rodriguez
(-0.48) Portuguese
(-1.54) Spanish
(-2.29) Dutch


> Mueller
(-0.76) Dutch
(-1.72) Czech
(-1.99) English


> Li
(-0.16) Vietnamese
(-2.36) Korean
(-3.03) Chinese


> Nakamura
(-0.01) Japanese
(-5.60) Portuguese
(-6.00) Arabic


> Smith
(-0.33) Scottish
(-2.19) German
(-2.92) Czech



## Conclusiones

La implementación con PyTorch Lightning ofrece las siguientes ventajas:

1. **Código más limpio y modular**: Separación clara de responsabilidades
2. **Mejor manejo de experimentos**: Logging automático y configuración fácil
3. **Escalabilidad**: Fácil adaptación a diferentes recursos de hardware
4. **Reproducibilidad**: Mejor control de semillas y configuraciones
5. **Funcionalidades avanzadas**: Early stopping, checkpointing, etc.
